taken from esgf-api example:
https://github.com/ESGF/esgf-compute-api/blob/master/examples/subset.ipynb    

In [ ]:
from owslib.wps import WebProcessingService

Get a new token from: https://aims2.llnl.gov/

Login with your ESGF OpendID (LLNL works).

In [ ]:
# token
api_key = 'TOKEN'


In [ ]:
# api_key should be in the HTTP headers
client = WebProcessingService('https://aims2.llnl.gov/wps/', verify=False, headers={'COMPUTE-TOKEN': api_key})

### GetCapabilities
https://aims2.llnl.gov/wps?service=WPS&version=1.0.0&request=GetCapabilities

In [ ]:
for p in client.processes:
    print(p.identifier)

### DescribeProcess
https://aims2.llnl.gov/wps?service=WPS&version=1.0.0&request=DescribeProcess&identifier=CDAT.subset

In [ ]:
proc = client.describeprocess(
    'CDAT.subset'
)
proc.identifier

In [ ]:
for inpt in proc.dataInputs:
    print(inpt.identifier, inpt.dataType)

In [ ]:
from owslib_esgfwps import Domain, Dimension

In [ ]:
d0 = Domain([
    Dimension('time', 227, 806, crs='indices'),
    Dimension('lat', 0, 90, crs='values'),
    Dimension('lon', 180, 360, crs='values'),
])


In [ ]:
print(d0.json)

In [ ]:
inputs = [('domain', d0)]

In [ ]:
from owslib_esgfwps import Variable

In [ ]:
# (122, 240, 480)
data_url = 'http://aims3.llnl.gov/thredds/dodsC/cmip5_css02_data/cmip5/output1/CMCC/CMCC-CM/decadal2005/mon/atmos/Amon/r1i2p1/tas/1/tas_Amon_CMCC-CM_decadal2005_r1i2p1_200511-201512.nc'

In [ ]:
v0 = Variable(uri=data_url, var_name='tas')

In [ ]:
v0.json

In [ ]:
inputs.append(('variable', v0))

In [ ]:
for inp in inputs:
    print(inp[1])

### Execute
https://aims2.llnl.gov/wps?service=WPS&version=1.0.0&request=Execute&identifier=CDAT.subset&api_key=TOKEN&DataInputs=domain={};variable={}

In [ ]:

url_templ = 'https://aims2.llnl.gov/wps?service=WPS&version=1.0.0&request=Execute&identifier=CDAT.subset&api_key={0}&DataInputs=domain={1};variable={2}'

exec_url = url_templ.format(api_key, d0.json, v0.json)
print(exec_url)

**TODO**: execution fails with
    
```
<ows:ExceptionText>Missing required input &quot;variable&quot; for process CDAT.subset</ows:ExceptionText>
```

In [ ]:
exec = client.execute(proc.identifier, inputs=inputs)

**TODO**: Execution fails ... reasons? No api_key etc ...

In [ ]:
exec.isComplete()

In [ ]:
exec.isSucceded()

In [ ]:
for output in exec.processOutputs:
    print(output.identifier, output.reference)